# ParamRef ref-first parameter system — tests & demos

This notebook is split into two main sections:

1. **Behavior tests**: assert the ref-first parameter semantics, idempotent updates, and event plumbing.
2. **Exploratory examples**: visual, user-friendly math exploration patterns using `SmartFigure`.


## Section 1 — Behavior tests

### 1. Environment and dependencies

This notebook expects:

- `sympy`
- `ipywidgets`
- the project installed or available on `sys.path`


In [ ]:
import sys
import importlib
from dataclasses import is_dataclass
from pathlib import Path
from typing import Any, Callable, Dict, Optional, Sequence

import sympy as sp
import ipywidgets as widgets
from IPython.display import display

_HAS_WIDGETS = True

ROOT = Path.cwd().resolve().parents[1]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from gu_toolkit.SmartFigure import SmartFigure
from gu_toolkit.ParamEvent import ParamEvent
from gu_toolkit.SmartSlider import SmartFloatSlider

print("Python:", sys.version)
print("sympy:", sp.__version__)
print("ipywidgets available:", _HAS_WIDGETS)


### 2. Adapter sanity checks

In [ ]:
fig = SmartFigure()
if _HAS_WIDGETS:
    display(fig)
else:
    print("NOTE: ipywidgets not available; skipping figure display.")

assert hasattr(fig, "parameter") and callable(fig.parameter), "Figure must have .parameter(...)"
assert hasattr(fig, "params"), "Figure must expose .params"

params = fig.params

assert hasattr(params, "__getitem__"), "fig.params must support __getitem__"
assert hasattr(params, "keys") and hasattr(params, "values"), "fig.params should look like a Mapping"

if ParamEvent is None:
    print("WARNING: ParamEvent could not be imported. Event normalization checks will be limited.")
else:
    if not is_dataclass(ParamEvent):
        print("WARNING: ParamEvent is not a dataclass (allowed, but some checks may be weaker).")

print("Adapter sanity checks passed.")


### 3. Shared utilities

In [ ]:
import contextlib

@contextlib.contextmanager
def expect_raises(exc_type):
    try:
        yield
    except exc_type:
        return
    except Exception as e:
        raise AssertionError(f"Expected {exc_type.__name__}, got {type(e).__name__}: {e}") from e
    raise AssertionError(f"Expected {exc_type.__name__} to be raised, but no exception occurred.")

class CounterCallback:
    def __init__(self):
        self.calls = 0
        self.events = []

    def __call__(self, event):
        self.calls += 1
        self.events.append(event)

def observe_call(ref, cb, *, fire=False):
    try:
        return ref.observe(cb, fire=fire)
    except TypeError:
        return ref.observe(cb)


### 4. Baseline: ref-first API

In [ ]:
fig = SmartFigure()
if _HAS_WIDGETS:
    display(fig)

a = sp.Symbol("a")

ref_a = fig.parameter(a)
assert ref_a is fig.params[a], "fig.parameter(a) should return the ref stored in fig.params[a]"
assert hasattr(ref_a, "value"), "ParamRef must expose .value"
assert hasattr(ref_a, "observe") and callable(ref_a.observe), "ParamRef.observe must exist and be callable"

print("Ref-first API baseline passed.")


### 4.1 Discovering supported attributes at runtime

Use `ref.capabilities()` to see which optional attributes are available on the underlying control.

In [ ]:
fig = SmartFigure()
if _HAS_WIDGETS:
    display(fig)

a = sp.Symbol("a")
ref = fig.parameter(a)
print(ref.capabilities())


### 5. Weak idempotency, defaults, and selective updates

Expected behavior:

- If a parameter does **not** exist, missing `min/max/step/value` are filled with defaults.
- If a parameter **does** exist, only the **specified** fields are updated.


In [ ]:
fig = SmartFigure()
if _HAS_WIDGETS:
    display(fig)

a = sp.Symbol("a")
ref = fig.parameter(a)

# Defaults on first creation
assert abs(ref.value - 0.0) < 1e-12
if hasattr(ref, "min"):
    assert abs(ref.min - (-1.0)) < 1e-12
if hasattr(ref, "max"):
    assert abs(ref.max - 1.0) < 1e-12
if hasattr(ref, "step"):
    assert abs(ref.step - 0.01) < 1e-12

# Selective updates on existing params (only specified fields should change)
value_before = ref.value
default_before = ref.default_value if hasattr(ref, "default_value") else None

fig.parameter(a, min=2.0, max=4.0)
if hasattr(ref, "min"):
    assert abs(ref.min - 2.0) < 1e-12
if hasattr(ref, "max"):
    assert abs(ref.max - 4.0) < 1e-12
assert abs(ref.value - value_before) < 1e-12
if hasattr(ref, "default_value"):
    assert abs(ref.default_value - default_before) < 1e-12

fig.parameter(a, step=0.5)
if hasattr(ref, "step"):
    assert abs(ref.step - 0.5) < 1e-12
if hasattr(ref, "default_value"):
    assert abs(ref.default_value - default_before) < 1e-12

fig.parameter(a, value=0.25)
assert abs(ref.value - 0.25) < 1e-12
if hasattr(ref, "default_value"):
    assert abs(ref.default_value - default_before) < 1e-12, "value updates must not change default_value"

print("Weak idempotency checks passed.")


### 6. `params` access semantics for value vs default_value

In [ ]:
fig = SmartFigure()
if _HAS_WIDGETS:
    display(fig)

a = sp.Symbol("a")
ref = fig.parameter(a, value=0.1)

# Setting value should not change default_value (when supported)
old_default = ref.default_value if hasattr(ref, "default_value") else None
ref.value = 0.9
assert abs(ref.value - 0.9) < 1e-12
if hasattr(ref, "default_value"):
    assert abs(ref.default_value - old_default) < 1e-12

# Setting default_value should not change current value (when supported)
if hasattr(ref, "default_value"):
    ref.default_value = -0.2
    assert abs(ref.default_value - (-0.2)) < 1e-12
    assert abs(ref.value - 0.9) < 1e-12
else:
    print("default_value not supported for this control; skipping default_value checks.")

print("params value/default_value semantics passed.")


### 7. Hook + observe plumbing (smoke test)

In [ ]:
fig = SmartFigure()
if _HAS_WIDGETS:
    display(fig)

a = sp.Symbol("a")
param_ref = fig.parameter(a)

# Attach an observer
cb = CounterCallback()
observe_call(param_ref, cb, fire=True)

# Change value
param_ref.value = param_ref.value + 0.5

assert cb.calls >= 1, "Observer did not fire."
print("Observer fired after parameter change.")


## Section 2 — Exploratory examples

These examples are intended to be **visual** and **user-friendly**. Each one displays the figure explicitly.


### Example 1: amplitude control for a sine wave

In [ ]:
x, a = sp.symbols("x a")

fig = SmartFigure(x_range=(-6, 6), y_range=(-2, 2))
fig.plot(x, a * sp.sin(x), parameters=[a], id="amp")
fig.title = r"$y = a\sin(x)$"

if _HAS_WIDGETS:
    display(fig)

fig.parameter(a, min=0.2, max=2.0, step=0.05, value=1.0)


### Example 2: mixing sine + cosine with two parameters

In [ ]:
x, a, b = sp.symbols("x a b")

fig = SmartFigure(x_range=(-6, 6), y_range=(-3, 3))
fig.plot(x, a * sp.sin(x) + b * sp.cos(x), parameters=[a, b], id="mix")
fig.title = r"$y = a\sin(x) + b\cos(x)$"

if _HAS_WIDGETS:
    display(fig)

fig.parameter(a, min=-2.0, max=2.0, step=0.1, value=1.0)
fig.parameter(b, min=-2.0, max=2.0, step=0.1, value=0.5)


### Example 3: slope-intercept line exploration

In [ ]:
x, m, b = sp.symbols("x m b")

fig = SmartFigure(x_range=(-5, 5), y_range=(-5, 5))
fig.plot(x, m * x + b, parameters=[m, b], id="line")
fig.title = r"$y = mx + b$"

if _HAS_WIDGETS:
    display(fig)

fig.parameter(m, min=-3.0, max=3.0, step=0.1, value=1.0)
fig.parameter(b, min=-3.0, max=3.0, step=0.1, value=0.0)
